# Footprint Generation
***
## Introduction
Generate footprints for Roman, HST and JWST apertures. 

With this approach, footprints are created from "exposures" which are specified with a target and an aperture (deifned by telescope, instrument, aperture name).

An Exposure can stand alone or be included in a pattern or observation.

An Exposure can be repeated at different (ideal) detector offsets by placing it in a generalized DitherPattern of regular repeated offsets or a CustomPattern where a list of custom offsets are specified.

For more complex groups of footprints, Exposures and Patterns can be added to Observations which can be added to Programs. Different position angles can be specified for different observations in the same program.

Program, Observation, Pattern and Exposure all support `get_exp_list()` which returns a list JSON representations of the contained exposures. 

### References
- [Roman coordinate systems](https://roman-docs.stsci.edu/data-handbook/wfi-data-levels-and-products/coordinate-systems)

### Acknowledgements
This notebook makes use of the [pysiaf package](https://github.com/spacetelescope/pysiaf) and is based on the [display_footprints MAST Notebook](https://github.com/spacetelescope/mast_notebooks/tree/main/notebooks/multi_mission/display_footprints) written by Brian McLean and Thomas Dutkiewicz.


### Imports

In [ ]:
from astroquery.mast import Mast
#from astropy.table import Table
from mast_aladin import AppSidecar, MastAladin
from mast_aladin.app import MastTable
#import pysiaf

from mast_aladin.utils.footprint_generator import (
   Exposure, Pattern, CustomPattern, DitherPattern, 
   Observation, Program, exp_list_to_table
)

### Start Aladin in a sidecar

In [ ]:
aladin = MastAladin(fov=1.0)
apps_initialized = AppSidecar.open(aladin, anchor='split-right')
AppSidecar.resize_all(800)

# Convenience method to display table data in aladin and MastTable 
# and move Aladin to target.
def display_in_aladin(exp_table, target, name='footprint_demo', show_table=True):
    aladin.target = f'{target.ra.deg} {target.dec.deg}'
    aladin.add_table(exp_table, ra_field="targ_ra", dec_field="targ_dec", name=name)
    if show_table:
        display(MastTable(exp_table))

## Examples

### Simple Roman exposure

The Exposure constructor defaults to using the Roman WFI "ALL" aperture, so we just need to specify a target if we want that.  Note that for Roman WFI ALL only, the individual component apertures get expanded into individual "exposures" in the exposure list we get.

Exposures themselves do not have a specified position angle (orientation).  When generating the exposure list with footprints, one can specify a pa.  Note that the Roman WFI detectors are oriented 60 degrees away from the telescope frame coordinates, so specifying a pa of 60 orients them as we are used to seeing.

In [ ]:
target = Mast.resolve_object("m101")
exp = Exposure(target.ra, target.dec)

# The optional pa (position angle) here rolls the telescope to make the WFI detectors "level". 
exp_table = exp_list_to_table(exp.get_exp_list(pa=60))

display_in_aladin(exp_table, target, name='Roman exp m101')

### Specifying other apertures
Using the `telescope`, `instrument` and `apertures` parameters, footprints can be generated for Roman, JWST and HST.  The available options for `telescope` are:
- `'roman'` allows these values for `instrument` and `aperture`:
  - `instrument`: ALL, WFI, CGI
  - `aperture`: ALL or individual apertures listed in [instrument documentation](https://roman-docs.stsci.edu/simulation-tools/additional-simulation-tools/pysiaf-for-roman)
- `'hst'` allows these values for `instrument` and `aperture`:
  - `instrument`: ALL, ACS, COS, FGS, NICMOS, STIS, WFC3
  - `aperture`: ALL or individual apertures listed in [instrument documentation](https://www.stsci.edu/hst/instrumentation/focus-and-pointing/fov-geometry)
- `'jwst'` allows these values for `instrument` and `aperture`:
  - `instrument`: ALL, FGS, MIRI, NIRCAM, NIRSPEC, NIRISS
  - `aperture`: ALL or individual apertures discussed generally in the [jwst-docs instrument-specific sections](https://jwst-docs.stsci.edu/#gsc.tab=0). See note below for more.

**Note**: It can be difficult to find the correct aperture names since the documentation doesn't always relfect the exact name in the SIAF files.  For example, The HAST ACS aperture WFC is called JWFC in the SIAF file.  All the current SIAF files (and unfortunately many older ones) can be found under [the prd_data directory in the pysiaf repository](https://github.com/spacetelescope/pysiaf/tree/main/pysiaf/prd_data).

#### Exposure offsets
Use the `x_off` and `y_off` optional arguments to offset the exposure from the target in ideal detector coordinates.

In [ ]:
# Show all the JWST NIRCAM apertures at m81.
target = Mast.resolve_object("m81")
exp = Exposure(target.ra, target.dec, 
               x_off=200, y_off=700,
               telescope='JWST', 
               instrument='NIRCAM', 
               aperture='ALL'
              )

exp_table = exp_list_to_table(exp.get_exp_list())
display_in_aladin(exp_table, target, name='NIRCam offset from m81')

### Dither Patterns
An exposure can be repeated in a grid by creating a `DitherPattern` with a specified number of rows and columns, yielding `num_rows * num_cols` total exposures.  

The position of the exposure at each pattern point is determined by offsets in detector ideal coordinates (arcsecs).  For each pattern point in a row, the pointing will be shifted on the sky in the direction of the row x and y offset.  Within a column, the additional column x and y offsets are applied.  Unspecified offsets default to 0, so to specify a rectangular grid, simply omit the `row_y_offset` and `col_x_offset`.

In [ ]:
# 3x2 pattern of JWST NIRCAM at m51
target = Mast.resolve_object('m51')
exp = Exposure(target.ra, target.dec,
               telescope='JWST', 
               instrument='NIRCAM', 
               aperture='NRCALL_FULL'
              )
pattern = DitherPattern(
    exp,
    num_rows=3, num_cols=2,
    row_x_offset=300, row_y_offset=10,
    col_x_offset=0, col_y_offset=130
)

exp_table = exp_list_to_table(pattern.get_exp_list(pa=90))
display_in_aladin(exp_table, target, name='NIRCam dither m51')

### Custom Patterns
A `CustomPattern` is similar to a `DitherPattern`, but instead of specifying grid parameters that get used to generate the offsets, the offsets are provided in a list, allowing total flexibility.

In [ ]:
# custom pattern that looks like a 3x2 grid with HST WFC3 UVIS at m51.
target = Mast.resolve_object('m51')
exp = Exposure(target.ra, target.dec,
               telescope='HST', 
               instrument='WFC3', 
               aperture='IUVIS'
              )
pattern = CustomPattern(exp,[
    [0, 0],
    [150, 0],
    [300, 0],
    [0, 150],
    [150, 150],
    [300, 150],
])

exp_table = exp_list_to_table(pattern.get_exp_list())
display_in_aladin(exp_table, target, name='WFC3 pattern m51')

### Putting Those together into a Full Program
By adding the above components to a virtual observing `Program`, a variety of footprints can be combined into a single bundle.

`Exposure`s and `Pattern`s can be added to `Observation`s which can be added to a `Program`.  Position angles can be specified for an `Observation` and that angle will be applied to all elements of the `Observation`.

In [ ]:
program = Program(1234)

# 3x2 JWST NIRCam pattern to cover outer nebula cloud
target = Mast.resolve_object('274.566487 -13.6011973')
obs = program.add_observation(Observation(150))
exp = Exposure(target.ra, target.dec,
               telescope='jwst', instrument='nircam',
               aperture='NRCALL_FULL')
obs.add_pattern(DitherPattern(
    exp,
    num_rows=3, num_cols=2,
    row_x_offset=100, row_y_offset=0,
    col_x_offset=0, col_y_offset=120
))

# Roman WFI coverage of main nebula
ngc6611 = target = Mast.resolve_object('ngc 6611')
obs = program.add_observation(Observation(210))
exp = Exposure(target.ra, target.dec, y_off=500)  # offset target to get best coverage
obs.add_pattern(DitherPattern(
    exp,
    num_rows=1, num_cols=5,
    col_x_offset=-300
))

# Look at the nearby "V* ES Ser" and "TYC 5689-418-1" with HST ACS WFC
obs = program.add_observation(Observation())
target = Mast.resolve_object('V* ES Ser')
obs.add_exposure(Exposure(target.ra, target.dec,
                          telescope='hst', instrument='ACS', aperture='JWFC'))
target = Mast.resolve_object('TYC 5689-418-1')
obs.add_exposure(Exposure(target.ra, target.dec,
                          telescope='hst', instrument='ACS', aperture='JWFC'))


exp_table = exp_list_to_table(program.get_exp_list())
display_in_aladin(exp_table, ngc6611, name='full program ngc6611')
aladin.fov = 1.2